# Genetik Algoritma ile Web Sunucusu Ayarları Optimizasyonu

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ferhancibik/genetik_optimizasyonu/blob/main/genetik_algoritma_optimizasyon.ipynb)

**Ad:** Ferhan

**Soyad:** Çıbık

**Okul Numarası:** 2312721038

**GitHub Repo Bağlantısı:** https://github.com/ferhancibik/genetik_optimizasyonu

---

## Senaryo 8: Web Sunucusu Ayarları

**Amaç Fonksiyonu:**
$$y = 5x_1 + 7x_2 - 0.1x_1^2 - 0.2x_2^2$$

**Değişkenler:**
- $x_1$: CPU çekirdeği sayısı → [2, 12]
- $x_2$: RAM miktarı (GB) → [4, 64]

**Kısıtlar:**
- $x_1 \times x_2 \leq 512$
- $x_1 \geq 4$


## 1. Kütüphaneler ve Parametreler


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random

np.random.seed(42)
random.seed(42)

X1_MIN, X1_MAX = 2, 12
X2_MIN, X2_MAX = 4, 64

POPULASYON_BOYUTU = 100
NESIL_SAYISI = 150
CAPRAZLAMA_ORANI = 0.8
MUTASYON_ORANI = 0.15
TURNUVA_BOYUTU = 5
ELIT_SAYISI = 2

print("Parametreler yüklendi!")
print(f"CPU: [{X1_MIN}, {X1_MAX}], RAM: [{X2_MIN}, {X2_MAX}] GB")
print(f"Popülasyon: {POPULASYON_BOYUTU}, Nesil: {NESIL_SAYISI}")


## 2. Amaç Fonksiyonu ve Kısıtlar


In [ ]:
def amac_fonksiyonu(x1, x2):
    """Sunucu performans skoru"""
    return 5*x1 + 7*x2 - 0.1*(x1**2) - 0.2*(x2**2)

def kisit_kontrol(x1, x2):
    """Kısıtları kontrol et"""
    if not (X1_MIN <= x1 <= X1_MAX and X2_MIN <= x2 <= X2_MAX):
        return False
    if x1 * x2 > 512 or x1 < 4:
        return False
    return True

print(f"Test: CPU=8, RAM=40 → Performans={amac_fonksiyonu(8, 40):.2f}")
print(f"Kısıt kontrolü: {kisit_kontrol(8, 40)}")


## 3. Birey Sınıfı

Her birey bir olası çözümü (CPU ve RAM değerleri) temsil eder.


In [ ]:
class Birey:
    def __init__(self, x1=None, x2=None):
        if x1 is None or x2 is None:
            self.x1, self.x2 = self._rastgele_uret()
        else:
            self.x1, self.x2 = x1, x2
        self.fitness = self._fitness_hesapla()
    
    def _rastgele_uret(self):
        """Kısıtları sağlayan rastgele değerler üret"""
        for _ in range(100):
            x1 = np.random.uniform(4, X1_MAX)
            x2_max = min(X2_MAX, 512 / x1)
            if x2_max >= X2_MIN:
                x2 = np.random.uniform(X2_MIN, x2_max)
                if kisit_kontrol(x1, x2):
                    return x1, x2
        return 4.0, 32.0
    
    def _fitness_hesapla(self):
        """Fitness hesapla, kısıt ihlalinde ceza"""
        if not kisit_kontrol(self.x1, self.x2):
            return -1000
        return amac_fonksiyonu(self.x1, self.x2)
    
    def __repr__(self):
        return f"Birey(CPU={self.x1:.2f}, RAM={self.x2:.2f}, Fit={self.fitness:.2f})"

print("Örnek bireyler:")
for i in range(3):
    print(f"  {i+1}. {Birey()}")


## 4. Genetik Operatörler

Seçilim, çaprazlama ve mutasyon fonksiyonları.


In [ ]:
def turnuva_secimi(populasyon, boyut=5):
    """Turnuva seçimi ile ebeveyn seç"""
    turnuva = random.sample(populasyon, boyut)
    return max(turnuva, key=lambda b: b.fitness)

def caprazlama(ebeveyn1, ebeveyn2):
    """Aritmetik çaprazlama"""
    alfa = np.random.uniform(0.3, 0.7)
    
    c1_x1 = alfa * ebeveyn1.x1 + (1-alfa) * ebeveyn2.x1
    c1_x2 = alfa * ebeveyn1.x2 + (1-alfa) * ebeveyn2.x2
    c2_x1 = (1-alfa) * ebeveyn1.x1 + alfa * ebeveyn2.x1
    c2_x2 = (1-alfa) * ebeveyn1.x2 + alfa * ebeveyn2.x2
    
    c1_x1 = np.clip(c1_x1, X1_MIN, X1_MAX)
    c1_x2 = np.clip(c1_x2, X2_MIN, X2_MAX)
    c2_x1 = np.clip(c2_x1, X1_MIN, X1_MAX)
    c2_x2 = np.clip(c2_x2, X2_MIN, X2_MAX)
    
    if not kisit_kontrol(c1_x1, c1_x2):
        c1_x1 = max(4, c1_x1)
        c1_x2 = min(c1_x2, 512 / c1_x1)
    
    if not kisit_kontrol(c2_x1, c2_x2):
        c2_x1 = max(4, c2_x1)
        c2_x2 = min(c2_x2, 512 / c2_x1)
    
    return Birey(c1_x1, c1_x2), Birey(c2_x1, c2_x2)

def mutasyon(birey, oran=0.15):
    """Gaussian mutasyon"""
    x1, x2 = birey.x1, birey.x2
    
    if np.random.random() < oran:
        x1 += np.random.normal(0, (X1_MAX - X1_MIN) * 0.1)
        x1 = np.clip(x1, X1_MIN, X1_MAX)
    
    if np.random.random() < oran:
        x2 += np.random.normal(0, (X2_MAX - X2_MIN) * 0.1)
        x2 = np.clip(x2, X2_MIN, X2_MAX)
    
    if not kisit_kontrol(x1, x2):
        x1 = max(4, x1)
        if x1 * x2 > 512:
            x2 = min(x2, 512 / x1)
    
    return Birey(x1, x2)

print("Genetik operatörler hazır!")


## 5. Ana Genetik Algoritma


In [ ]:
def genetik_algoritma():
    """Ana genetik algoritma döngüsü"""
    populasyon = [Birey() for _ in range(POPULASYON_BOYUTU)]
    
    en_iyi_gecmis = []
    ortalama_gecmis = []
    
    for nesil in range(NESIL_SAYISI):
        populasyon.sort(key=lambda b: b.fitness, reverse=True)
        
        en_iyi_gecmis.append(populasyon[0].fitness)
        ortalama_gecmis.append(np.mean([b.fitness for b in populasyon]))
        
        if (nesil + 1) % 20 == 0 or nesil == 0:
            print(f"Nesil {nesil+1}: En İyi Fitness = {populasyon[0].fitness:.2f}")
        
        yeni_pop = populasyon[:ELIT_SAYISI]
        
        while len(yeni_pop) < POPULASYON_BOYUTU:
            ebeveyn1 = turnuva_secimi(populasyon, TURNUVA_BOYUTU)
            ebeveyn2 = turnuva_secimi(populasyon, TURNUVA_BOYUTU)
            
            if np.random.random() < CAPRAZLAMA_ORANI:
                cocuk1, cocuk2 = caprazlama(ebeveyn1, ebeveyn2)
            else:
                cocuk1 = Birey(ebeveyn1.x1, ebeveyn1.x2)
                cocuk2 = Birey(ebeveyn2.x1, ebeveyn2.x2)
            
            cocuk1 = mutasyon(cocuk1, MUTASYON_ORANI)
            cocuk2 = mutasyon(cocuk2, MUTASYON_ORANI)
            
            yeni_pop.append(cocuk1)
            if len(yeni_pop) < POPULASYON_BOYUTU:
                yeni_pop.append(cocuk2)
        
        populasyon = yeni_pop
    
    populasyon.sort(key=lambda b: b.fitness, reverse=True)
    return populasyon[0], en_iyi_gecmis, ortalama_gecmis


## 6. Algoritmayı Çalıştır


In [ ]:
print("="*60)
print("GENETİK ALGORİTMA ÇALIŞIYOR...")
print("="*60)
print()

en_iyi, en_iyi_gecmis, ortalama_gecmis = genetik_algoritma()

print()
print("="*60)
print("SONUÇLAR")
print("="*60)
print()
print(f"🏆 EN İYİ ÇÖZÜM:")
print(f"   CPU: {en_iyi.x1:.2f} çekirdek")
print(f"   RAM: {en_iyi.x2:.2f} GB")
print(f"   Performans Skoru: {en_iyi.fitness:.2f}")
print()
print(f"📊 KISIT KONTROLÜ:")
carpim = en_iyi.x1 * en_iyi.x2
print(f"   CPU × RAM = {carpim:.2f} (≤ 512) {'✓' if carpim <= 512 else '✗'}")
print(f"   CPU = {en_iyi.x1:.2f} (≥ 4) {'✓' if en_iyi.x1 >= 4 else '✗'}")
print(f"   Kullanım Oranı: %{(carpim/512)*100:.1f}")


## 7. Sonuçların Görselleştirilmesi


In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
nesiller = range(1, len(en_iyi_gecmis) + 1)
plt.plot(nesiller, en_iyi_gecmis, 'g-', linewidth=2, label='En İyi Fitness')
plt.plot(nesiller, ortalama_gecmis, 'b--', linewidth=1.5, label='Ortalama Fitness')
plt.xlabel('Nesil', fontsize=12)
plt.ylabel('Fitness Değeri', fontsize=12)
plt.title('Fitness Evrimi', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
if len(nesiller) > 50:
    son_50 = list(nesiller[-50:])
    plt.plot(son_50, en_iyi_gecmis[-50:], 'g-', linewidth=2, label='En İyi')
    plt.plot(son_50, ortalama_gecmis[-50:], 'b--', linewidth=1.5, label='Ortalama')
else:
    plt.plot(nesiller, en_iyi_gecmis, 'g-', linewidth=2, label='En İyi')
    plt.plot(nesiller, ortalama_gecmis, 'b--', linewidth=1.5, label='Ortalama')
plt.xlabel('Nesil', fontsize=12)
plt.ylabel('Fitness Değeri', fontsize=12)
plt.title('Yakınsama Analizi (Son 50 Nesil)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('fitness_evrimi.png', dpi=300, bbox_inches='tight')
plt.show()

print("Grafik 'fitness_evrimi.png' olarak kaydedildi.")


## 8. Detaylı Analiz


In [ ]:
print("="*60)
print("DETAYLI ANALİZ")
print("="*60)
print()

ilk_fitness = en_iyi_gecmis[0]
son_fitness = en_iyi_gecmis[-1]
iyilesme = son_fitness - ilk_fitness
iyilesme_yuzde = (iyilesme / abs(ilk_fitness)) * 100 if ilk_fitness != 0 else 0

print(f"⚡ PERFORMANS:")
print(f"   İlk Nesil: {ilk_fitness:.2f}")
print(f"   Son Nesil: {son_fitness:.2f}")
print(f"   İyileşme: {iyilesme:.2f} ({iyilesme_yuzde:.1f}%)")
print()

son_10_ort = np.mean(en_iyi_gecmis[-10:])
son_10_std = np.std(en_iyi_gecmis[-10:])

print(f"🎯 YAKINSAMA:")
print(f"   Son 10 Nesil Ortalama: {son_10_ort:.2f}")
print(f"   Standart Sapma: {son_10_std:.4f}")

if son_10_std < 0.01:
    print(f"   Durum: Çok iyi yakınsadı! ✓")
elif son_10_std < 0.1:
    print(f"   Durum: İyi yakınsadı ✓")
else:
    print(f"   Durum: Hala iyileşiyor...")

print()
print("="*60)
